## 6. Web UI
The Web UI provides very basic graphical interaction to the planning scene.
* **6.1 Instance tab**  
  - highlight, add/delete, edit, apply
* **6.2 Binding tab**  
  - highlight, edit
  
-------------------- deprecated ------------------
* **6.3 Mark tab**
  - highlight
  - MarkEnv, MarkObj
* **6.4 Planning tab**  
  - Conditions
    - Initialize, Plan
  - **PlanList**
    - Replay, Execute
* **6.5 Setting tab**
  - Robot
  - MotionPlanner
  - TaskPlanner
  
#### Required environment
* box1
* goal

## set running directory to project source

In [1]:
import os
import sys
import numpy as np
import time
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## Prepare pipeline

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder
from pkg.controller.combined_robot import CombinedRobot
from pkg.controller.robot_config import RobotConfig, RobotType

from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.aruco.marker_config import get_aruco_map
from pkg.detector.camera.kinect import Kinect
from pkg.detector.camera.realsense import RealSense
from pkg.detector.detector_interface import DetectionLevel

INDY_IP = "192.168.21.6"
PANDA_HOST_IP = "192.168.21.14" ## The host computer is the computer that runs control program
PANDA_ROBOT_IP = "192.168.21.13" ## The robot has it's own IP

crob = CombinedRobot(
    robots_on_scene=[
        RobotConfig(0, RobotType.indy7, ((0,-0.3,0), (0,0,0)),
                    INDY_IP),
        RobotConfig(1, RobotType.panda, ((0,0.3,0), (0,0,0)),
                    "{}/{}".format(PANDA_HOST_IP, PANDA_ROBOT_IP))
    ], 
    connection_list=[False, False])

stereo = ArucoStereo(aruco_map=get_aruco_map(), 
                     camera_list=[Kinect(), RealSense()])

stereo.initialize()

time.sleep(1) # Let the camera have some time to get stable
stereo.calibrate()
print("Detector initialized")

s_builder = SceneBuilder.instance(detector=stereo)
# deprecated: s_builder.reset_reference_coord(ref_name="floor")

xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
ptems = s_builder.add_poles(
    {"cam0": stereo.ref_coord_inv[:3,3], 
     "cam1":np.matmul(stereo.ref_coord_inv, stereo.T_c12)[:3,3]},
    color=(0.6,0.6,0.6,0.2))
gscene.show_pose(crob.home_pose)

gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])
track = gtem_dict["track"]

gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])
s_builder.give_clearance(track, gtem_dict.values())

from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool

gscene.create_safe(GEOTYPE.MESH, "indy0_gripper_vis", link_name="indy0_tcp", dims=(0.1,0.1,0.1),
                   center=(0,0,0), rpy=(0,0,np.pi/2), color=(0.1,0.1,0.1,1), display=True, fixed=True, collision=False, 
                   uri="package://my_mesh/meshes/stl/indy_gripper_asm2_res.STL", scale=(1,1,1))

gscene.create_safe(GEOTYPE.BOX, "indy0_gripper", link_name="indy0_tcp", dims=(0.06,0.08,0.06),
                   center=(0,0,0.04), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

gscene.create_safe(GEOTYPE.CYLINDER, "indy0_finger1", link_name="indy0_tcp", dims=(0.03,0.03,0.095),
                   center=(0.006,0.045,0.1), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

gscene.create_safe(GEOTYPE.CYLINDER, "indy0_finger2", link_name="indy0_tcp", dims=(0.03,0.03,0.095),
                   center=(-0.006,0.045,0.1), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

gscene.create_safe(GEOTYPE.CYLINDER, "indy0_finger3", link_name="indy0_tcp", dims=(0.03,0.03,0.095),
                   center=(0.006,-0.045,0.1), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

gscene.create_safe(GEOTYPE.CYLINDER, "indy0_finger4", link_name="indy0_tcp", dims=(0.03,0.03,0.095),
                   center=(-0.006,-0.045,0.1), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5), display=True, fixed=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)
pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="track", gname="track", _type=PlacePlane, point=None)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))

from pkg.planning.constraint.constraint_subject import BoxObject
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=BoxObject)

from pkg.planning.pipeline import PlanningPipeline
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.task.rrt import TaskRRT

ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(MoveitPlanner(pscene))
ppline.set_task_planner(TaskRRT(pscene))

connection command:
indy0: False
panda1: False
Device configuration: 
	color_format: 3 
	(0:JPG, 1:NV12, 2:YUY2, 3:BGRA32)

	color_resolution: 5 
	(0:OFF, 1:720p, 2:1080p, 3:1440p, 4:1536p, 5:2160p, 6:3072p)

	depth_mode: 3 
	(0:OFF, 1:NFOV_2X2BINNED, 2:NFOV_UNBINNED,3:WFOV_2X2BINNED, 4:WFOV_UNBINNED, 5:Passive IR)

	camera_fps: 2 
	(0:5 FPS, 1:15 FPS, 2:30 FPS)

	synchronized_images_only: False 
	(True of False). Drop images if the color and depth are not synchronized

	depth_delay_off_color_usec: 0 ms. 
	Delay between the color image and the depth image

	wired_sync_mode: 0
	(0:Standalone mode, 1:Master mode, 2:Subordinate mode)

	subordinate_delay_off_master_usec: 0 ms.
	The external synchronization timing.

	disable_streaming_indicator: False 
	(True or False). Streaming indicator automatically turns on when the color or depth camera's are in use.


Start streaming
Detector initialized
ros_node already initialized somewhere else
Please create a subscriber to the marker
publication 

## Start Web UI
* Run below cell and access to UI on web browser at <IP>:8050

In [3]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
generate table - Geometry
generate table - Handle
generate table - Binder
generate table - Object


#### 6.1 Instance tab
Instance tab lists geometries and planning objects in the scene.  
* **highlight**
  - On **Geometry** and **Object** panel, click listed items and check if corresponding geometry item is highlighted in RVIZ.
* **add/delete**
  - On **Geometry** panel, click "x" icon on the left side of a listed item and see if the item is removed from the scene.
  - On **Object** panel, click "x" icon on the left side of a listed item.
    - Refresh the screen and check if corresponding handles are removed from the *Handle* panel in **Binding** tab
* **edit**
  - On **Geometry** panel, click any listed item field, edit and press enter. Check if the geometry is changed in RVIZ.
    - Please be careful to keep the format of the item field contents.
* **apply**
  - "Disp" field is not applied instantly when edited. Change one item's "Disp" field from True to False or from False to True, click apply and check if  the item disappears or appears.

#### 6.2 Binding tab
Binding tab lists all action points in the planning scene.  
To test this section, re-start this notebook to recover the items that are removed in 6.1  
* **highlight**
  - On **Handle** and **Binder** panel, click listed items and check if corresponding geometry item is highlighted in RVIZ.
  - This time the corresponding axis will also be visualized.
* **edit**
  - Edit "Point" and "RPY" field of some items in **Handle** and **Binder** panel, see if the change is applied in the RVIZ.